In [2]:
import numpy as np
import os
import re
import sys

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Users\amanda\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\amanda\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\amanda\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\amanda\AppData\Roaming\P

In [3]:
def sample(preds, temp=1.0):
    '''
    Helper function for sampling from softmax with different temperatures.
    
    Inputs:
    preds: output of softmax function
    temp: temperature to scale by
    '''
    preds = np.asarray(preds).astype('float64')
    exp_preds = preds**(1/temp)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [4]:
# load data
text = open(os.path.join(os.getcwd(), 'data/shakespeare.txt')).read()
    
# remove sonnet numbers and convert to lowercase
text = re.sub(r'[0-9]+', '', text) 
text = text.lower()

# create mapping of unique chars to integers as well as reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(text) # total length of dataset
n_vocab = len(chars) # number of unique characters

In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 40
dataX = []
dataY = []

# use sliding window approach
for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_sequences = len(dataX)

In [6]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_sequences, seq_length, 1))

# normalize data to range (0, 1)
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
# define the LSTM model 
# (single layer with 200 units, followed by dense output layer)
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))

In [8]:
# load the network weights from best checkpoint
#filename = "weights-improvement-20-1.7396.hdf5"
filename = "checkpoints/weights-improvement-72-1.1011.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [68]:
# set seed for emissions
seed = "shall i compare thee to a summer's day?\n"
seed_to_int = [char_to_int[char] for char in seed]

# generate emissions
pattern = seed_to_int

temps = [0.01, 0.1, 0.25, 0.75, 1.5]    
for temp in temps:    
    print('Temperature', temp)
    
    # generate characters
    newlines = 0
    cap = True
    while newlines < 14:
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)[0]

        # sample according to temperature
        idx = sample(prediction, temp)

        result = int_to_char[idx]
        pattern.append(idx)
        pattern = pattern[1:len(pattern)]
        
        # output result 
        if cap:
            sys.stdout.write(result.capitalize())
            cap = False
        else:
            sys.stdout.write(result)
        sys.stdout.flush()
        
        if result == '\n':
            newlines += 1
            cap = True
            
    print()

Temperature 0.01
That thou mooe tiat at men hoow here ho deed.
Nhe ail my verse and mea sime's sore make soolne,
When touuh ae oithilg the dioucr torr:
Oo both brascd thtu art a lov thou crt,
But the hesh thee as thou wilt bestar aro,
For then iy leaied that mencs thie,
And lo hrane sreek the side the thmeees of thee,
The may uhes flues of thee me oney decda


                   
When i thal whx lnte in then i wes cerpod,
What thou dass tron and wruld thou stuly toone:
So suay that live aed nos (harnsg fyen heve,

Temperature 0.1
What yhuhont to-hr conf that i lo nertiess beare,
Then thou art flvstsess tiell ast hear,
Whechnli the swmengt of thes world ay thee,
The west bonouert of thy sirl't pace,
The beangrt your soue nove' my self anooe.
And thetefore world co thnu wilt bestar aered,
How hasp the beauty of thy beyter beyter,
Which ii the thieks seetse of thy becuty,
Which sooe i sey hooe that thou havest beaney
Bh tusthd toee with thene ooet dormess,
Which wiol not be thet the owesi